In [ ]:
import numpy as np
from scipy.signal import convolve2d 
from os import path, getcwd, system, getenv
from astropy.io import fits
from time import perf_counter
import urllib.request
import os

import matplotlib
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import glob
import matplotlib.gridspec as gridspec


from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.backends.backend_pdf import PdfPages

import sys,inspect
currentdir = path.dirname(path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = path.dirname(currentdir)
sys.path.append(parentdir)
# from deconv import cube_deconv
# from ppxf_wrap import ppxf_wrap

# import warnings

# if True:
#     print('Set warnings.filterwarnings as "ignore"')
#     print('Warning Message will not be printed')
#     warnings.filterwarnings("ignore")

from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy import constants
import pandas as pd

# import plotly.express as px
from astroquery.mast import Observations


from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from astropy.nddata.utils import Cutout2D

from astropy.wcs import WCS
import astropy

from urllib.error import HTTPError
from astroquery.skyview import SkyView
import pickle


computer = 1
maindir = "C:/Users/maxri/Desktop/Classes 4-1/Research/fuse"
obn = 0
    


for ii in range(47,81,1):
    obn = ii
    #1
    #Create Object List/
    #Save to Object Tables
    
    skydowni ="./images_redOb{}".format(obn); 
    catalogdir = "C:/Users/maxri/Desktop/Classes 4-1/Research/data/VI_129_fuse.dat.fits"
    catalog = fits.open(catalogdir)

    catheader = catalog[0]; catlist = catalog[1];

    dflist = pd.DataFrame(catlist.data)



    catalog.close()


    obs=dflist.query("Class == {} and Avail=='y'".format(obn));
    
    if len(obs) == 0:
        continue

    dflist

#     #Reading Index
#     indexdir = f"{maindir}/tbls/{obn}index.txt"
#     indexdir

#     ipd = pd.read_table(indexdir, header = None)




    obsn1 = []
    for i in obs.iterrows():
        b = [i[1][1].split()[0],i[1][2], i[1][3]] 
        obsn1.append(b)



    obsn1

    s1 = maindir+"/tbls/Objects{}.tbl".format(obn); 

    np.savetxt(s1,obsn1,fmt="%s",delimiter = " ")



    #2
    #Skyview Query by Object name or Coordinate, reads in the from Object table. 
    #Image query block

    size = 1600

    width_val = u.Quantity(size, unit = 'arcsec')
    height_val =  u.Quantity(size, unit = 'arcsec')

    t3 = []
    # if computer == 2:
    #     t = pd.read_table("C:/Users/maxri/OneDrive/Desktop/Classes 4-1/Research/fuse/objs2.tbl",header=None)
    # if computer == 1:
    #     t = pd.read_table("C:/Users/maxri/Desktop/Classes 4-1/Research/fuse/tbls/objs2.tbl",header=None)


        #Read selected class object table
    if computer == 1:
        try:
            t2 = pd.read_table(s1,header=None)
        except:
            continue
    




    # for i in t.values:
    #     t3.append(i[0])


    coordsn = [];t4=[]


    for i in t2.values:
        b = i[0].split()
        c = SkyCoord(b[1],b[2], unit=u.deg)
        coordsn.append(c)
        t4.append(b[0])

    coordsn


    objnamesn = t4; objnames = objnamesn;


    obstables2 = []; coords = [];

    #pixels initially 300 x 300
    #Initial Skyview Query by name

    surveys = ['DSS', 'DSS1 Blue', 'DSS1 Red', 'DSS2 Red', 'DSS2 Blue', 'DSS2 IR'] #use surveyindex = 3 Dss2red

    surveyindex = 3


    for i in range(len(objnamesn)):
            paths = [];
            try:
                paths = SkyView.get_images(position = coordsn[i], 
                           survey=surveys[surveyindex],width = width_val,height = height_val,pixels = [1000,1000])
                hdr = paths[0][0].header
                a = hdr['HISTORY'][33].split()
                cd= [float(a[2]),float(a[3])]
                print(objnamesn[i],surveys[surveyindex])
                c = SkyCoord(cd[0],cd[1], unit=u.deg)
                coords.append(c)

            except HTTPError:
                print("No Result for ", objnamesn[i], surveys[surveyindex])
            except TypeError:
                print("No Result for ", objnamesn[i], surveys[surveyindex])
            except IndexError:
                print("No Result for ", objnamesn[i], surveys[surveyindex])
            obstables2.append(paths)


    objnames = objnamesn

    result = pd.DataFrame(obstables2)
    obstables2, result, objnamesn,coords
    #objnamesn, coordsn


    #3
    #Skyview Download Images Code Block, to same convention as before
    #Create Download directory too

    pt = os.path.join(maindir, skydowni[2:])
    try:
        os.mkdir(os.path.normpath(pt))
        print("Created images_red directory")
    except:
        print("Directory already exists")

    result;
    coords
    for i in range(len(result)):
        a = surveys[surveyindex].split() #Survey
        b = objnames[i].split()
        name = b[0] #+ b[1]     #OBjName
        if i < 9:
            fname = "000{}_{}_{}_{}_{}".format(i+1,coordsn[i].ra.deg,coordsn[i].dec.deg,a[0]+a[1],name)
        else:
            fname = "00{}_{}_{}_{}_{}".format(i+1,coordsn[i].ra.deg,coordsn[i].dec.deg,a[0]+a[1],name)
        #print(fname)
        try:
            result[0][i][0].writeto(skydowni+ "/"+fname+".fits", output_verify='exception', overwrite=True, checksum=False)
        except:
            print(f"No result for {fname}")


    #4
    #MAin FUSE DOWNLOAD BLOCK
    def set_rcparams_prop(amp=1, fs=22, tickamp=None):
        fs=18
        if tickamp==None:
            tickamp=amp
        tickmajorwidth=2.5*amp
        tickminorwidth=2.5*amp
        axeslw=2.5*amp
        tmjs=6*tickamp
        tmns=tmjs/3.5*2
        plt.rcParams['axes.titlesize']=fs
        plt.rcParams['axes.labelsize']=fs
        plt.rcParams['xtick.labelsize']=fs
        plt.rcParams['ytick.labelsize']=fs
        plt.rcParams['axes.linewidth'] = axeslw
        plt.rcParams['xtick.major.width'] = tickmajorwidth
        plt.rcParams['ytick.major.width'] = tickmajorwidth
        plt.rcParams['xtick.minor.width'] = tickminorwidth
        plt.rcParams['ytick.minor.width'] = tickminorwidth
        plt.rcParams['xtick.major.size'] = tmjs
        plt.rcParams['ytick.major.size'] = tmjs
        plt.rcParams['xtick.minor.size'] = tmns
        plt.rcParams['ytick.minor.size'] = tmns

    #Download Method
    # obs_tables=[]
    # for i in range(len(objnames)):
    #     print(objnames[i]) #objectname=objname[i] vs coordinates=coordsn[i] Object Name vs Coordinate Query
    #     obs_tables.append(Observations.query_criteria(obs_collection='FUSE',coordinates=coordsn[i], radius=".5 deg"))


    #Pickle Method obs_tables
    pickledir = maindir + "/Pickles/" + "Ob{}".format(obn)
    with open(pickledir, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        obs_tables = pickle.load(f)
        f.close

    #Pickle Method Product List
    pickledir2 = maindir + "/Pickles/" + "pl{}".format(obn)
    with open(pickledir2, 'rb') as g:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        pl = pickle.load(g)
        g.close

    #Main Fuse Download
    fnames=[]


    #download files
    if computer == 1:
        down_dir=maindir + "/temp"
    if computer == 2:
        down_dir = 'C:/Users/maxri/OneDrive/Desktop/Classes 4-1/Research/fuse/temp'

    #Download block 2
    for i in range(len(obs_tables)):
    # for i in [6]:
        otb=obs_tables[i]
        urls=pl[i]['dataURI']
        rr=[x.find('00000all2ttag') for x in urls]
        dpl=pl[i][np.array(rr)>0]
        print(urls)
        
        stopper = 0
        try:
            manifest = Observations.download_products(dpl, download_dir=down_dir)
        except Warning:
            stopper = stopper +1
        print(obs_tables[i]['target_name'])
        print(manifest)

        rr=[x.find('00000all4ttag') for x in urls]
        dpl=pl[i][np.array(rr)>0]
        print(urls)
        try:
            manifest = Observations.download_products(dpl, download_dir=down_dir)
        except Warning:
            stopper = stopper +1
        print(obs_tables[i]['target_name'])
        print(manifest)

    #TO HEREE
        rr=[x.find('00000all3ttag') for x in urls]
        dpl=pl[i][np.array(rr)>0]
        print(urls)
        try:
            manifest = Observations.download_products(dpl, download_dir=down_dir)
        except Warning:
            stopper = stopper +1
        if stopper == 3:
            continue
        print(obs_tables[i]['target_name'])
        print(manifest)
    #    needs documentation

        exptimes=obs_tables[0]['t_exptime']
    #     print(exptimes)
        cond=exptimes>5000
        


    #5
    # Pointings download/Creating organised Data Frames
    # Slowest Block currently
    # Not sure if some parts could be removed.

    if computer == 1:
        down_dir=maindir + '/temp/'
    if computer == 2:
        down_dir = 'C:/Users/maxri/OneDrive/Desktop/Classes 4-1/Research/fuse/temp/'


    df=[];

    # for i in range(1):

    for i in range(len(obs_tables)):
        otb=obs_tables[i]
        urls=pl[i]['dataURI']
        rr=[x.find('00000all4ttag') for x in urls]
        dpl=pl[i][np.array(rr)>0]
        objname=objnames[i]
        for j in range(len(dpl)):
    #     for j in range(1):
            fdir=down_dir+'mastDownload/FUSE/'+dpl[j]['obs_id']+'/'+dpl[j]['productFilename']
            hdu=fits.open(fdir)
            hdr=hdu[0].header
            sri=pd.Series({'objname':objname, 
                           'targname':hdr['TARGNAME'],
                           'ra':hdr['RA_TARG'],
                           'dec':hdr['DEC_TARG'],
                           'pa':hdr['APER_PA'],
                           'aperture':hdr['APERTURE'],
                           'programid':hdr['PRGRM_ID'],
                           'exptime':hdr['OBSTIME'],
                           'obsmode':hdr['INSTMODE'],
                           'fdir':down_dir+'mastDownload/FUSE/'+dpl[j]['obs_id']+'/'+dpl[j]['productFilename']
                          })
            print(hdr['APERTURE'])
            df.append(sri)

        rr=[x.find('00000all2ttag') for x in urls]
        dpl=pl[i][np.array(rr)>0]
        objname=objnames[i]
        for j in range(len(dpl)):
            fdir=down_dir+'mastDownload/FUSE/'+dpl[j]['obs_id']+'/'+dpl[j]['productFilename']
            hdu=fits.open(fdir)
            hdr=hdu[0].header
            sri=pd.Series({'objname':objname, 
                           'targname':hdr['TARGNAME'],
                           'ra':hdr['RA_TARG'],
                           'dec':hdr['DEC_TARG'],
                           'pa':hdr['APER_PA'],
                           'aperture':hdr['APERTURE'],
                           'programid':hdr['PRGRM_ID'],
                           'exptime':hdr['OBSTIME'],
                           'obsmode':hdr['INSTMODE'],
                           'fdir':down_dir+'mastDownload/FUSE/'+dpl[j]['obs_id']+'/'+dpl[j]['productFilename']
                          })
            print(hdr['APERTURE'])
            df.append(sri)


    df = pd.DataFrame(df)    

    # hdr        
    #     manifest = Observations.download_products(dpl, download_dir=down_dir)
    #     print(manifest)


    #6
    #Max & Mean exposure calc

    tcut=5000
    tcut=2000
    ndf=df[df['exptime']>tcut]

    df2=pd.DataFrame()

    j1 = []
    for i in range(len(objnames)):
        print(objnames[i])
        adf=ndf[ndf['objname']==objnames[i]]
        adf.index=np.arange(len(adf))
        if len(adf) ==0:
            continue
        sri=pd.Series({
            'objname':objnames[i],
            'ra':adf.loc[0,'ra'],
            'dec':adf.loc[0,'dec'],
            'npt':len(adf),
            'maxexp':np.max(adf.loc[:,'exptime'].values),
            'meanexp':np.mean(adf.loc[:,'exptime'].values),
        })
        df2=df2.append(sri, ignore_index=True)
        j1.append(i)
    df2


    # Table: 
    # Galaxy, Program ID, N pointings (exp >5000s), RA, DEC, Max Exp, Mean Exp
    # *note on aperture
    # *RA DEC unit check




    #7
    #Setup functions for plots, roudnings and rotating
    def fstr(x):
        return x

    def ff6(x):
        return '%.6f' % x


    def ff4(x):
        return '%.4f' % x

    def ff1(x):
        return '%.1f' % x

    def ff2(x):
        return '%.2f' % x

    def fint(x):
        return '%i' % x

    def detection_check(x):
        if x>3:
            return 'Y'
        else:
            return 'N'


    def rec_rotate(cx, cy, wx, wy, ang_deg): #angle: clocowise rot from 12h
        ang_rad=np.radians(ang_deg)
        wd=(wx**2+wy**2)**0.5
        ang_diagonal=np.arctan(wy/wx)
        ang_new=ang_diagonal+ang_rad
        rx=cx-wd/2*np.cos(ang_new)
        ry=cy-wd/2*np.sin(ang_new)
        return rx, ry

    df2['randec']=df2['ra'].apply(ff4)+', '+df2['dec'].apply(ff4)
    df2['totalexp']=df2['meanexp']*df2['npt']

    # coords = SkyCoord(df2['ra'], df2['dec'], frame='icrs', unit=(u.hourangle, u.deg))
    coords = SkyCoord(df2['ra'], df2['dec'], frame='icrs', unit=(u.deg, u.deg))
    df2['l']=coords.galactic.l.degree
    df2['b']=coords.galactic.b.degree
    # df2['rvel']=[269,850,308,500,133,465,241,-39,841,259,1127,757,976,470,610,528] #Radial Velocities
    rvel = [2790,3400,-12.2,1647.9,4925,-42.5, -6.2,-101.1,850.0,-106.5,8.9,200,-46.6,200,-4.3,42.1]
    rvel = np.repeat(rvel,math.ceil(len(objnames))/16 + 1,axis=0)
    rvel = rvel[0:len(j1)]
    df2['rvel']=rvel #Test Radial Velocities
    #df2['rvel']=[269]

    df2['lnb']=df2['l'].apply(ff4)+', '+df2['b'].apply(ff4)

    df2['owave']=1031.9*(1+df2['rvel']/299790)

    # fmts=[fstr, ff4, ff4, fint, fint, fint]
    # fmts=[fstr, ff4, ff4, fint, fint, fint,ff1]
    fmts=[fstr, fstr, fint, fint, fint,ff1]
    # fmts=[fstr, fstr, fint, fint, fint, fint]
    # ridx1=[8,9,3,4,5,6,7,0,1,2]
    latex_text=df2.to_latex(index=False, header=False,
                            formatters=fmts,
    #              columns=['objname','ra','dec','npt','maxexp','totalexp'])
    #              columns=['objname','l','b','npt','maxexp','totalexp'])
    #              columns=['objname','l','b','npt','maxexp','totalexp','owave'])
                 columns=['objname','lnb','npt','maxexp','totalexp','owave'])
    #              columns=['objname','ra','dec','npt','maxexp','meanexp'])
    print(latex_text)

    #print('\hline')
    print(' & & & '+fint(np.sum(df2['npt']))+' & &  '+fint(np.sum(df2['npt']*df2['meanexp'])))

    # df2.to_csv('adap_2022_targets.csv')


    #8
    #Reading in downloaded fits and labeling
    if computer == 1:
        file_dir=maindir +'/images_red'      #manual
        file_dir = maindir +  skydowni[1:]     #Skyview
    if computer == 2:
        file_dir = 'C:/Users/maxri/OneDrive/Desktop/Classes 4-1/Research/fuse/images_red'      #manual
        file_dir = 'C:/Users/maxri/OneDrive/Desktop/Classes 4-1/Research/fuse/images_redTEST'  #Skyview

    file_path=np.array(glob.glob(file_dir+'**/*fits'))
    file_path[0].split('_')
    # file_path[0].split('/')

    fdf=pd.DataFrame()
    for i in range(len(file_path)):
        fp=file_path[i]
        print(fp)
        sri=pd.Series({'ra':fp.split('_')[2],
                       'dec':fp.split('_')[3],
                       'file_path':file_dir+'/'+fp.split('/')[-1],
                       'filename':fp.split('/')[-1],
                      })
        fdf=fdf.append(sri, ignore_index=True)


    print(fdf)

    print(fdf.filename[0])
    print(fdf.file_path[0])

    fdf.file_path[0]


    corrdir  = maindir + f"/corrtbls/corr{obn}.txt"
    corrs = pd.read_table(corrdir,header = None)
    #9
    #Main Plotting Block
#     idex = []
#     for j2 in range(len(objnames)):
#         if ipd[0][j2] == 1:
#             idex.append(j2)


    def p16(pg):
        set_rcparams_prop(amp=1.15)

        rr=1
        aa=6
        wrs=np.array([1.8, 0.2, 1.2, 0.05])
        hrs=np.array([1])
        fig = plt.figure(figsize=(20, 30))
        fig = plt.figure(figsize=(25, 25))
        plt.title("Class {} Plots".format(obn),fontdict={'fontsize': 50})
        ncol=4
        nrow=4
        gs1 = gridspec.GridSpec(nrow, ncol, figure=fig, 
        #                         width_ratios=wrs, height_ratios=hrs,
                                hspace=0.05, wspace=0.05
                               )
        pidxs=np.arange(ncol*nrow)
        axs=[]
        for i in range(len(pidxs)):
            ax=fig.add_subplot(gs1[i // ncol, i%ncol])
            axs.append(ax)


        cut_widths=np.array([
            240,240,240,240,
            240,240,240,240,
            240,240,240,240,
            240, 240,240,240,
        ])/3600*50





        fidx = np.zeros(len(obs_tables))


        minsep=0.5
        file_sc=SkyCoord(fdf.ra, fdf.dec, frame='icrs', unit='deg')    
        
        
        


        for i in range(0,16,1):
            ax=axs[i]
            j5 = i + 16*pg
            try:
                newi = j1[j5]
                objname=objnames[newi]
            except:
                break
                
                
            print(objname, f"Index  {newi}")
            sdf=df.loc[df['objname']==objname,:]
            sdf.index=np.arange(len(sdf))
            ra0, dec0=sdf.loc[0,'ra'], sdf.loc[0,'dec']
        #     print(ra0, dec0)
            c = SkyCoord(ra0, dec0, frame='icrs', unit='deg')

            cond=np.array(file_sc.separation(c))<minsep
            match_df=fdf[cond]
            match_df.index=np.arange(np.sum(cond))

            if computer ==1:
                if obn <10:
                    fits_file=match_df.loc[fidx[newi],'file_path'][0:45] + match_df.loc[fidx[newi],'file_path'][64:] #Skyview
                else:
                    try:
                        fits_file=match_df.loc[fidx[newi],'file_path'][0:44] + match_df.loc[fidx[newi],'file_path'][64:] #Skyview
                    except:
                        continue
            print(fidx[i])
            print('number of matched DF', len(match_df))
            hdu=fits.open(os.path.normpath(fits_file))
            hdr=hdu[0].header
            oiwcs=astropy.wcs.WCS(header=hdr) 
            
            g = corrs[0][newi].split()
            ccorr = []; b = [];
            stop =0
#             if g[3] == "NONE":
#                 stop = 1
            if stop == 1:
                 continue
            else:
                for t9 in range(int(len(g)/3 - 1)):
                    b = b + [g[4 + 3*t9]] + [g[5 + 3*t9]]
                    
            print(fits_file)
            cmap=cm.gray_r
            origin='lower'

            oim=hdu[0].data

            #Calculate offset between ra0 and owics, cc and cutout
            sky = oiwcs.pixel_to_world(500, 500)
            dra = sky.ra-c.ra; ddec =  sky.dec - c.dec  ;
            raoff = dra.deg/cut_widths[1]; decoff = ddec.deg/cut_widths[1];

            print(raoff,decoff, "Offsets")



            ra0, dec0=sdf.loc[0,'ra']+cut_widths[i]*raoff, sdf.loc[0,'dec']+cut_widths[i]*decoff

            print(ra0, dec0 , "Skyview Coords")

            cc = SkyCoord(ra0, dec0, frame='icrs', unit='deg')

            print(oiwcs,cc)
            print('npt',len(sdf))
            img_cutout_obj=Cutout2D(oim, cc, 
                               u.Quantity(cut_widths[i], unit='deg'),
                               wcs=oiwcs,
                            mode='trim',
        #                     mode='partial',
                           )

            im=img_cutout_obj.data
            iwcs=img_cutout_obj.wcs

            img=ax.matshow(im, 
                       cmap=cmap, 
                       origin=origin, interpolation='none', 
        #                zorder=1,
        #               alpha=alpha, zorder=zorder, aspect=aspect, 
        #                norm=matplotlib.colors.LogNorm(vmin=vmin, vmax=vmax)
                          )
        #     print(objname)
            
            # ADD NGC Pointings
            for m in range(int(len(b)/2)):
                    t10 = SkyCoord(b[0 + 2*m],b[1+2*m],unit = u.deg)
                    pointx, pointy=iwcs.world_to_pixel(t10)
                    patches = matplotlib.patches.Circle((pointx, pointy), radius=15, color='red')
                    ax.add_patch(patches)
                    print(pointx,pointy, "NGC Coords")

            for j in range(len(sdf)):
                jra, jdec=sdf.loc[j,'ra'], sdf.loc[j,'dec']        
                c_obj=SkyCoord(jra, jdec, 
                                                  unit='deg',
                                                  frame='icrs')
                if sdf.loc[j,'aperture']=='MDRS':
                    sep=u.Quantity(-90.18-118.07, unit='arcsec')
                    c_obj=c_obj.directional_offset_by(u.deg*(sdf.loc[j,'pa']+180), sep)
                    print('MDRS!!!')

                sep=u.Quantity(90.18+118.07, unit='arcsec')
                m_obj=c_obj.directional_offset_by(u.deg*(sdf.loc[j,'pa']+180), sep)
                sep=u.Quantity(-10.27+118.07, unit='arcsec')
                h_obj=c_obj.directional_offset_by(u.deg*(sdf.loc[j,'pa']+180), sep)

                objs=[c_obj, m_obj, h_obj]
                xws=[30,4,1.25]
                yws=[30,20,20]
                clrs=['m','c','r']

                for k in range(3):
                    pcent_x, pcent_y=iwcs.world_to_pixel(objs[k])
                    psm=iwcs.pixel_scale_matrix
                    pscale=(psm[0,0]**2+psm[0,1]**2)**0.5
            #         print('pscale',pscale)
                    pxwidth=xws[k]/3600/pscale
                    pywidth=yws[k]/3600/pscale
        #             edgecolor='m'
                    edgecolor=clrs[k]
            #         edgecolor='k'
                    sqlw=2


                    rot_ang=0
                    rot_ang=sdf.loc[j,'pa']
                    recx, recy=rec_rotate(pcent_x, pcent_y, pxwidth, pywidth, rot_ang)
                    rec=matplotlib.patches.Rectangle((recx, recy,), pxwidth, pywidth, alpha=1, 
                                                     angle=rot_ang,
                                                     zorder=20,
                #     rec=matplotlib.patches.Rectangle((pcent_x-pxwidth/2,pcent_y-pywidth/2), pxwidth, pywidth, alpha=1, 
                #                                      angle=(-180),
                #                                   facecolor='w', fill=True)
                                            linewidth=sqlw, edgecolor=edgecolor, linestyle='-', fill=False)
                    #                                               linewidth=1.5, edgecolor='w', linestyle=esty[k-1], fill=False)
                    art=ax.add_artist(rec)
                    
               
                
                    
                ax.text(0.98,0.02,objname,
                        horizontalalignment='right',
                        verticalalignment='bottom',
                        transform=ax.transAxes,
                        size=30
                       )
                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_xticklabels([])
                ax.set_yticklabels([])
                ax.set_aspect('equal')
            
        #         pcent_x, pcent_y=iwcs.world_to_pixel(m_obj)
        #         psm=iwcs.pixel_scale_matrix
        #         pscale=(psm[0,0]**2+psm[0,1]**2)**0.5
        # #         print('pscale',pscale)
        #         pxwidth=4/3600/pscale
        #         pywidth=20/3600/pscale
        #         edgecolor='c'
        # #         edgecolor='k'
        #         sqlw=3


        #         rot_ang=0
        #         rot_ang=sdf.loc[j,'pa']
        #         recx, recy=rec_rotate(pcent_x, pcent_y, pxwidth, pywidth, rot_ang)
        #         rec=matplotlib.patches.Rectangle((recx, recy,), pxwidth, pywidth, alpha=1, 
        #                                          angle=rot_ang,
        #                                          zorder=20,
        #     #     rec=matplotlib.patches.Rectangle((pcent_x-pxwidth/2,pcent_y-pywidth/2), pxwidth, pywidth, alpha=1, 
        #     #                                      angle=(-180),
        #     #                                   facecolor='w', fill=True)
        #                                 linewidth=sqlw, edgecolor=edgecolor, linestyle='-', fill=False)
        #         #                                               linewidth=1.5, edgecolor='w', linestyle=esty[k-1], fill=False)
        #         art=ax.add_artist(rec)

        csp=0.0
        fig.subplots_adjust(wspace=csp, hspace=csp)

        # aa=cc.separation(c)
        # np.array(Angle(aa, unit=u.deg))<0.5
        #match_df
        try:
            hdu.close()
        except:
            return(fig)


        #fig.savefig(fname,dpi=256, bbox_inches='tight')
        return(fig)

    s4 = maindir + "/tbls/Classes.txt"

    ntbl = pd.read_table(s4, header = None)

    name = ntbl[0][obn].split()[1:]
    name = " ".join(name)
    name

    p = f"{obn}{name}+NGC.pdf"

    with PdfPages(p) as pdf:
        for j in range(0, math.ceil(len(objnames)/16 )):
            print(j)
            fig = p16(0+j)
            pdf.savefig(fig)

    plt.close(fig)




In [ ]:
 for l in range(len(corrs[0])):
    g = corrs[0][l].split()
    ccorr = []; b = [];
    if g[3] == "NONE":
        print("none found")
    else:
        for t9 in range(int(len(g)/3 - 1)):
            b = b + [g[4 + 3*t9]] + [g[5 + 3*t9]]
    print(b)
b

In [ ]:
g = corrs[0][0].split()
int(len(g)/3)

In [ ]:
corrs

In [ ]:
b

In [ ]:
 cut_widths=np.array([
            240,240,240,240,
            240,240,240,240,
            240,240,240,240,
            240, 240,240,240,
        ])/3600*8 
cut_widths 
